<br>

<h1>
    <center>
        Estimating $\pi$ using a Qauntum Computer
    </center>
</h1>

## Table of Contents


0. [Define Problem](#0.-Define-Problem)
1. [Import Libraries](#)
2. [Estimate $\pi$ using Monte Carlo Simulation](#)
3. [Estimate $\pi$ using Taylor Series Expansion](#) 
4. [Estimate $\pi$ using Quantum Accelerated Monte Carlo Simulation](#) 
    * Generate Random 8-bit Numbers
    * Normalize Random Numbers Between [0,1]
    * Measure Circle vs. Square Ratio
5. [Analysis of Performance](#)
    * Accuracy
    * Speed
6. [Conclusion - Which is better?](#)

### 0. Define Problem


![thing]()

In [1]:
# import visulaization tool
import seaborn as sns
import matplotlib.pyplot as plt

# import numerical processing tool
import numpy as np

# data handeling
import pandas as pd

# measure time/duration 
import time

# import quantum simulation tools
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import BasicAer, execute
import pylatexenc

In [2]:
def CreateCircuit(bits):
    """
    Input:
        bits | int | defines how many q-bits the circuit involves
    Output:
        circuit | qiskit.circuit | establish a circuit to randomly generate 8-bit binary numbers
        circuit_diagram | matplotlib.figure | figure that depicts the q-bit to c-bit figure
    """    
    
    # define quantum register | classical register
    q = QuantumRegister(bits);  c = ClassicalRegister(bits)
    
    # define circuit
    circuit = QuantumCircuit(q, c)
    
    # use for loops to add q-bits and hadamar gates to the existing circuit
    for j in range(bits): circuit.h(q[j])
    
    # add measurements to transfer from q-bit to classical-bit
    circuit.measure(q, c)
    
    # draw circuit
    circuit_drawing = circuit.draw(output='mpl')
    
    # return circuit and drawing
    #return (circuit, circuit_drawing)
    return circuit

In [11]:
def GenerateRandomNumbers_QC(circuit, sample_size):
    """
    Input:
        circuit | qiskit.circuit | establish a circuit to randomly generate 8-bit binary numbers
        sample_size | int | number of iterations to generate 8-bit numbers
    Output:
        numbers | list(int) | list of binary numbers generated by the circuit
    """   
    
    # generate binary numbers
    binary_numbers = execute(
        experiments = circuit,
        backend = BasicAer.get_backend('qasm_simulator'),
        shots = sample_size,
        memory = True
    )
    
    # convert binary numbers to integers
    numbers = np.array([int(i, 2) for i in binary_numbers.result().get_memory()])
    
    return numbers

In [4]:
def NormalizeNumbers(numbers):
    """
    Input:
        numbers | list(int) | list of binary numbers generated by the circuit
    Output:
        normalized_values | list(float) | list of normalized numbers between [0,1]
    """
    
    return [(num - min(numbers)) / (max(numbers) - min(numbers)) for num in numbers]

In [5]:
def ColorContext(x):
    """
    Input:
        x | list(float) | list of randomly generated numbers
    Output:
        color | str | color assigned to a point based on relation to circle 
    """
    
    if x < 1:
        color = 'Inside'
    else:
        color = 'Outside'
        
    return color    

In [6]:
def MeasurePi(x, y):    
    """
    Input:
        x | list(float) | list of randomly generated numbers
        y | list(float) | list of randomly generated numbers
    Ouptut:
        pi_approximation | float | approximate value of pi
        population | int | sample size previously defined
        df | pd.dataframe | common data used to combines all attributes
    """
    
    df = (
        pd
        .DataFrame(
            data = list(zip(x, y)),
            columns = ['x', 'y']
        )
        .assign(
            cicular_translation = lambda x: np.sqrt(x['x']**2 + x['y']**2),
            color = lambda x: x['cicular_translation'].map(ColorContext)
        )
        .drop('cicular_translation', axis = 1)
    )
    
    
    pi_approximation = (4*len(df.query("color == 'Inside'"))) / len(df)
    population = len(df)
    
    return pi_approximation, population, df

In [ ]:
# establish circute
bits = 8
circuit = CreateCircuit(bits = bits)

# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

# perform iterations    
for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6)]:

    print(sample_size)
        
    # measure start time
    start = time.time()   
    
    # generate x values
    x = NormalizeNumbers(GenerateRandomNumbers_QC(circuit = circuit, sample_size = sample_size))
    # generate y values
    y = NormalizeNumbers(GenerateRandomNumbers_QC(circuit = circuit, sample_size = sample_size))
    
    # create MC 
    mc_values = np.sqrt(np.square(x) + np.square(y))
    pi_approximation = 4*len(mc_values[mc_values < 1]) / len(x)
    
    # measure stop time
    stop = time.time()
    
    # record keeping
    duration.append(stop-start)        
    pi_approximation_list.append(pi_approximation)
    population_list.append(sample_size)

100
1000
10000
100000
1000000


In [ ]:
display(duration, pi_approximation_list, population_list)

In [ ]:
QuantumComputing_Performance = (
    pd
    .DataFrame(
        data = list(zip(pi_approximation_list, population_list, duration)),
        column = ['pi_approximatation', 'population', 'duration']
    )
    .assign(
        pi_actual = [np.pi],
        error = lambda x: abs(x['pi_approximatation'] - x['pi_actual'])
    )   
)

Approximating $\pi$ as a Series:


$$\frac{\pi}{4} = \frac{(-1)^{n}}{2n-1}$$

$$\therefore$$ $$\pi = 4\frac{(-1)^{n}}{2n-1}$$